In [1]:
from dataclasses import dataclass
import os
import datetime
import pickle
import numpy as np
from transformers import AutoModelForMultipleChoice
from torch.nn import CrossEntropyLoss
import random
from torch.utils.data import DataLoader, Dataset
import gc
import datetime
from tqdm import tqdm
import numpy as np
from numpy.random import Generator,default_rng
import re
from sklearn.model_selection import StratifiedKFold,KFold
from scipy.stats import rankdata
import torch
import transformers
import time
from torch.optim import Adam
from sklearn.metrics import log_loss
from torch.cuda.amp import GradScaler, autocast
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader

import pandas as pd
from torch.utils.data import TensorDataset
from transformers import BertTokenizer, RobertaTokenizer, BartTokenizer, AutoTokenizer
from sklearn.metrics import precision_score, recall_score, f1_score
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler,WeightedRandomSampler
from torch.nn import CrossEntropyLoss, MSELoss
from transformers import (
    BertForSequenceClassification,
    AdamW,
    get_linear_schedule_with_warmup,
)
from transformers import (
    RobertaForSequenceClassification,
    BartForSequenceClassification,
    AutoModelForSequenceClassification,
)
from tqdm.auto import tqdm


## Proprocessing - Tokenize the text data for BERT

In [4]:
def get_tensor_dataset(df, tokenizer):

    # Tokenize all of the sentences and map the tokens to thier word IDs.
    input_ids = []
    token_type_ids = []
    attention_masks = []
    labels = []
    topic_ids = []

    for count, item in tqdm(
        enumerate(
            zip(
                df["GameId"],
                df["bylevel_color_context"],  # df["nonspatial_color_context"],
                df["InputInstruction"],
                df["IsInstructionClear"],
            )
        ),
        total=len(df),
        desc="Tokenizing data",
    ):
        z, w, x, y = item
        encoded_dict = tokenizer.encode_plus(
            w,
            x,
            add_special_tokens=True,
            max_length=max_seq_length,
            padding="max_length",  # use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'`
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt",
        )

        input_ids.append(encoded_dict["input_ids"])

        if "token_type_ids" in encoded_dict:
            token_type_ids.append(encoded_dict["token_type_ids"])

        attention_masks.append(encoded_dict["attention_mask"])
        labels.append(y)

        topic_ids.append(z)

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels)
    return TensorDataset(input_ids, attention_masks, labels)


In [8]:
from sklearn.model_selection import train_test_split

data_path='public_data/clarifying_questions_train.csv'
df = pd.read_csv(data_path, sep=",")
df = pd.read_csv("public_data/clarifying_questions_train_split.csv")
df_test = pd.read_csv("public_data/clarifying_questions_test.csv")

  

In [27]:
train_posranker_selection = pd.read_pickle("../my_data/full_db_posranker_selection.pk")
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")
# tokenizer.convert_ids_to_tokens(tokenized_swag['train'][0]['input_ids'][2])
# tokenized_swag['train'][0]['input_ids']

from dataclasses import dataclass
from transformers.tokenization_utils_base import (
    PreTrainedTokenizerBase,
    PaddingStrategy,
)
from typing import Optional, Union
import torch


@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)]
            for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        # print(batch)
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/felipe/anaconda3/envs/iglu2/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:434: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [28]:
df = pd.read_csv("public_data/clarifying_questions_w_context.csv")
train_posranker_selection = train_posranker_selection.merge(
    df[["GameId", "nonspatial_color_context",'bylevel_color_context']],how='left'
)
dc = DataCollatorForMultipleChoice(tokenizer)


In [47]:
games_info = {}

for key, df in train_posranker_selection.groupby("GameId"):
    qlist = df["proposed_q"].to_list()
    realq = df["qtrue"].iloc[0]
    games_info[key] = {
        "rank_score": df["score"].to_list(),
        "clarifying_questions": df["ClarifyingQuestion"].to_list(),
        "proposed_qid": qlist,
        "real_q": realq,
        "instruction": df['bylevel_color_context'].iloc[0]
        + tokenizer.sep_token
        + df["InputInstruction"].iloc[0],
        #'context':,
        "label": qlist.index(realq) if realq in qlist else None,
    }


In [48]:
def get_questions_around_label(realq_id,qlist,num=25):
    top = min(len(qlist),realq_id+(num//2))
    return qlist[top-num:top]



In [50]:
block_colour_name_map = {
    # voxelworld's colour id : iglu colour id
    0: "air",
    1: "blue",
    6: "yellow",
    2: "green",
    4: "orange",
    5: "purple",
    3: "red",
}

In [51]:
original_colours =[val for key,val in block_colour_name_map.items() if val !='air']

In [52]:

gen = default_rng(seed=42)
gen.permutation(original_colours)
cur_per = gen.permutation(original_colours)
no_per = {key:key for key in original_colours}
per_dict ={ key:val for key,val in zip(original_colours,cur_per)}

In [60]:

def replace_full_words(text, dic):
    for i, j in dic.items():
        text = re.sub(r"\b%s\b" % i, j, text)
        # r"\b%s\b"% enables replacing by whole word matches only
    return text

In [63]:
npqs=120
def generate_augmented_games_info():
    
    original_colours =[val for key,val in block_colour_name_map.items() if val !='air']
    no_per = {key:key for key in original_colours}
    c_i_q_sets =[]
    for i in range(5):
        if i==0:
            cur_per=no_per
            
            
        else:
            ar_per = gen.permutation(original_colours)
            cur_per ={ key:val for key,val in zip(original_colours,ar_per)}
            
            
        games_info = {}
        
        for key, df in train_posranker_selection.groupby("GameId"):
            qlist = df["proposed_q"].to_list()
            realq = df["qtrue"].iloc[0]
            clariq = df["ClarifyingQuestion"].to_list()
            rank_score =df["score"].to_list()

            original_true_index = qlist.index(realq) 
            if original_true_index<npqs:
                qlist = qlist[:npqs]
                rank_score = rank_score[:npqs],
                clariq = clariq[:npqs]
            else:
                if np.random.choice([False,True]):
                    left_qs = np.random.randint(10,70)
                    qlist = qlist[:left_qs]+get_questions_around_label(original_true_index,qlist,num=npqs-left_qs)
                    rank_score = rank_score[:left_qs]+get_questions_around_label(original_true_index,rank_score,num=npqs-left_qs)
                    clariq = clariq[:left_qs]+get_questions_around_label(original_true_index,clariq,num=npqs-left_qs)
                else:

                    left_qs = np.random.randint(10,70)
                    qlist = get_questions_around_label(original_true_index,qlist,num=npqs-left_qs) + qlist[:left_qs]
                    rank_score = get_questions_around_label(original_true_index,rank_score,num=npqs-left_qs) + rank_score[:left_qs]
                    clariq = get_questions_around_label(original_true_index,clariq,num=npqs-left_qs) + clariq[:left_qs]

            games_info[key] = {
                "rank_score": rank_score,
                "clarifying_questions":[replace_full_words(q,cur_per) for q in clariq],
                "proposed_qid": qlist,
                "real_q": realq,
                "instruction": replace_full_words(df['bylevel_color_context'].iloc[0]
                + tokenizer.sep_token
                + df["InputInstruction"].iloc[0],cur_per),
                #'context':,
                "label": qlist.index(realq) if realq in qlist else None,
            }

        c_i_q_sets.append(games_info)
    
    return c_i_q_sets
        


    

In [64]:
c_i_q_sets = generate_augmented_games_info()
games_info = c_i_q_sets[2]

In [70]:
games_with_target = {
    game: features
    for game, features in games_info.items()
    if features["label"] is not None
}


In [71]:
num_pos_questions = npqs


def preprocess_function(examples):
    first_sentences = [
        examples["instruction"]
    ] * num_pos_questions  # [[context] * len(num_pos_questions) for context in examples["instruction"]]

    second_sentences = examples["clarifying_questions"]

    # first_sentences = sum(first_sentences, [])
    # second_sentences = sum(second_sentences, [])

    tokenized_examples = tokenizer(
        first_sentences, second_sentences, truncation=True, padding=True
    )

    return {
        k: [v[i : i + num_pos_questions] for i in range(0, len(v), num_pos_questions)]
        for k, v in tokenized_examples.items()
    }


In [72]:
preproc = preprocess_function(games_info["CQ-game-1117"])


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [74]:
os.environ["TOKENIZERS_PARALLELISM"] = 'false'

In [75]:
def generate_games_with_target(c_i_q_sets):
    import multiprocess
    augmented_games_with_targets=[]
    for data in c_i_q_sets:
     

        dataset_games_with_targets = list(data.values())

        dataset_games_tok = [
            {k: item[k] for k in ["clarifying_questions", "instruction"]}
            for item in dataset_games_with_targets
        ]

        with multiprocess.Pool(processes=4) as pool:

            res = pool.map(preprocess_function, dataset_games_tok)

            # pool.join()
        res[0]
        # @TOD batch it
        # tokenized_swag = swag.map(preprocess_function, batched=True)
        

        for i in range(len(res)):

            dataset_games_with_targets[i].update(res[i])

        augmented_games_with_targets.append(dataset_games_with_targets)   

    return augmented_games_with_targets 

In [76]:
augwt =  generate_games_with_target(c_i_q_sets)

In [81]:
class MultiPleChoiceRankerAugmentedTrainDataset(Dataset):
    def __init__(self,tokenized_array_list):

        self.data = tokenized_array_list
        self.columns =  ['label', 'input_ids', 'token_type_ids', 'attention_mask']
    def __len__(self):
        return len(self.data[0])

    def __getitem__(self, index):

        data_idx =np.random.randint(0,len(self.data))

        row =self.data[data_idx][index]

        for col in ['input_ids','token_type_ids', 'attention_mask']:

            if len(row[col])==1:
                row[col] = row[col][0]#[torch.Tensor(val) for val in row[col][0] ]
            #row[col] = torch.Tensor(row[col])

        #print(row)
        
        
        return {col: row[col] for col in self.columns}
   

In [82]:
class MultiPleChoiceRankerDataset(Dataset):
    def __init__(self,tokenized_array):

        self.data = tokenized_array
        self.columns =  ['label', 'input_ids', 'token_type_ids', 'attention_mask']
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):

        row =self.data[index]

        for col in ['input_ids','token_type_ids', 'attention_mask']:

            if len(row[col])==1:
                row[col] = row[col][0]#[torch.Tensor(val) for val in row[col][0] ]
            #row[col] = torch.Tensor(row[col])

        #print(row)
        
        
        return {col: row[col] for col in self.columns}
   

In [83]:
dataset = MultiPleChoiceRankerAugmentedTrainDataset(augwt)


In [85]:
dl = DataLoader(
    dataset, collate_fn=DataCollatorForMultipleChoice(tokenizer), batch_size=4
)


In [95]:
loss_fn = CrossEntropyLoss()


In [98]:
class CFG:
    wandb = True
    competition = "PPPM"
    _wandb_kernel = "nakama"
    debug = False
    apex = True
    print_freq = 100
    num_workers = 4
    model = "microsoft/deberta-v3-large"
    scheduler = "linear"  #'cosine' # ['linear', 'cosine','constant','plateau']
    batch_scheduler = True
    num_cycles = 1.0
    num_warmup_steps = 0
    epochs = 5
    encoder_lr = 7e-5#5e-5  # lr 5e-5 accumulation 4 batch 8 wd 0.003produces overfittin in the first epch
    decoder_lr = 3e-5  # 3e-5# 8e-6
    min_lr = 1e-7
    eps = 1e-8
    betas = (0.9, 0.999)
    batch_size = 36
    fc_dropout = 0.2
    target_size = 1
    max_len = 320
    weight_decay = 0.003#0.01  # 0.002#0.015#0.005#0.001#0.00005
    gradient_accumulation_steps = 1
    max_grad_norm = 60  # 200#500 #10#100#1000#150#1000
    seed = 42
    n_fold = 5
    trn_fold = [0, 1, 2, 3, 4]
    train = True


In [105]:
EPOCHS = 5
accumulation_steps =8

In [107]:
def get_optimizer_params(model, encoder_lr, decoder_lr, weight_decay=0.0):
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {
            "params": [
                p
                for n, p in model.named_parameters()
                if not any(nd in n for nd in no_decay)
            ],
            "lr": encoder_lr,
            "weight_decay": weight_decay,
        },
        {
            "params": [
                p
                for n, p in model.named_parameters()
                if any(nd in n for nd in no_decay)
            ],
            "lr": encoder_lr,
            "weight_decay": 0.0,
        },
        # {'params': [p for n, p in model.named_parameters() if "model" not in n],
        #     'lr': decoder_lr, 'weight_decay':  weight_decay}
    ]
    return optimizer_parameters


def get_scheduler(cfg, optimizer, num_train_steps):
    # if cfg.scheduler == 'linear':
    #     scheduler = transformers.get_linear_schedule_with_warmup(
    #         optimizer, num_warmup_steps=0.2 * num_train_steps, num_training_steps=num_train_steps
    #     )
    if cfg.scheduler == "linear":
        scheduler = transformers.get_linear_schedule_with_warmup(
            optimizer, num_warmup_steps=0, num_training_steps=num_train_steps
        )
    elif cfg.scheduler == "cosine":
        scheduler = transformers.get_cosine_schedule_with_warmup(
            optimizer,
            num_warmup_steps=0.2 * num_train_steps,
            num_training_steps=int(num_train_steps * 1.1),
            num_cycles=cfg.num_cycles,
        )
    elif cfg.scheduler == "constant":
        scheduler = transformers.get_constant_schedule_with_warmup(
            optimizer, num_warmup_steps=0.3 * num_train_steps
        )

    elif cfg.scheduler == "plateau":
        scheduler = transformers.ReduceLronPlateau(
            optimizer, num_warmup_steps=0.3 * num_train_steps
        )

    return scheduler


#scheduler = get_scheduler(CFG, optimizer, num_train_steps)

In [109]:
epochs= 5
lr=CFG.encoder_lr
model_hug = "microsoft/deberta-v3-base"
CFG.weight_decay=0.005

In [119]:
class AWP:
    def __init__(
        self,
        model,
        optimizer,
        adv_param="weight",
        adv_lr=1e-2,
        adv_eps=9e-4,
        start_epoch=0,
        adv_step=1,
        scaler=None
    ):
        self.model = model
        self.optimizer = optimizer
        self.adv_param = adv_param
        self.adv_lr = adv_lr
        self.adv_eps = adv_eps
        self.start_epoch = start_epoch
        self.adv_step = adv_step
        self.backup = {}
        self.backup_eps = {}
        self.scaler = scaler
        self.loss = torch.nn.CrossEntropyLoss()
    def attack_backward(self, x, y, attention_mask,epoch):
        if (self.adv_lr == 0) or (epoch < self.start_epoch):
            return None

        self._save() 
        for i in range(self.adv_step):
            self._attack_step() 
            with torch.cuda.amp.autocast():
                tr_logits = self.model(x,attention_mask)['logits']
                adv_loss =  self.loss(tr_logits,y)
                adv_loss = adv_loss.mean()
            self.optimizer.zero_grad()
            self.scaler.scale(adv_loss).backward()
            
        self._restore()

    def _attack_step(self):
        e = 1e-6
        #e=1e-3
        #e=1e-3
        for name, param in self.model.named_parameters():
            if param.requires_grad and param.grad is not None and self.adv_param in name:
                norm1 = torch.norm(param.grad)
                norm2 = torch.norm(param.data.detach())
                if norm1 != 0 and not torch.isnan(norm1):
                    r_at = self.adv_lr * param.grad / (norm1 + e) * (norm2 + e)
                    param.data.add_(r_at)
                    param.data = torch.min(
                        torch.max(param.data, self.backup_eps[name][0]), self.backup_eps[name][1]
                    )
                # param.data.clamp_(*self.backup_eps[name])

    def _save(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad and param.grad is not None and self.adv_param in name:
                if name not in self.backup:
                    self.backup[name] = param.data.clone()
                    grad_eps = self.adv_eps * param.abs().detach()
                    self.backup_eps[name] = (
                        self.backup[name] - grad_eps,
                        self.backup[name] + grad_eps,
                    )

    def _restore(self,):
        for name, param in self.model.named_parameters():
            if name in self.backup:
                param.data = self.backup[name]
        self.backup = {}
        self.backup_eps = {}


In [135]:
model_name="deberta_cv_full_recall_25_25_env_on_full_db120_augmentcolorper_wd5e-3_AWP_continued_awp2_3"

In [139]:
def train_ranker(model_name,model,dl,dl_val,scheduler,
        optimizer,
        loss_fn,
        epochs,
        lr,
        fold,):


    

    scaler = torch.cuda.amp.GradScaler()
    
    awp = AWP(model,
      optimizer,
      # adv_lr=args.adv_lr,
      # adv_eps=args.adv_eps,
      start_epoch=1,
      scaler=scaler
         )


    general_val_loss = []
    for epoch in range(EPOCHS):
        #  if DEBUG and (epoch>0): break
        start_time = time.time()
        avg_loss = 0.0
        model.train()
        tbar = tqdm(dl)
        loss_list = []
        val_loss_list = []
        model.train()

        # freeze_embd(net,i)
        total_rr=[]
        optimizer.zero_grad(set_to_none=True)
        for step, batch in enumerate(tbar):

            batch = {key: part.to(device) for key, part in batch.items()}
            # input_ids = data['ids']
            # input_masks = data['mask']
            # tag_ids = data['tag_ids']
            # targets = data['targets']#.long().view(-1).cuda()

            # input_ids,input_masks = checkpoint(net,**dict(ids=input_ids,mask=input_masks))

            with torch.cuda.amp.autocast():
                pred = model(batch["input_ids"], attention_mask=batch["attention_mask"])[
                    "logits"
                ]
                # pred = checkpoint(net,(input_ids,input_masks))

                # loss = loss_fn(pred.softmax(dim=1), targets)
                loss = loss_fn(pred, batch["labels"])

            # loss.backward()

            scaler.scale(loss).backward()
            loss_list.append(loss.detach().cpu().item())
            if epoch > -1:
                if loss <2.3: #0.78
                    awp.attack_backward(x=batch["input_ids"],y=batch['labels'],attention_mask=batch["attention_mask"],epoch=epoch)
                del loss
                gc.collect()
                torch.cuda.empty_cache()
                # awp.attack_backward(x=input_ids,y=targets,attention_mask=input_masks,tag_ids=tag_ids,epoch=epoch)

            else:

                del loss

            # grad_norm = torch.nn.utils.clip_grad_norm_(net.parameters(), CFG.max_grad_norm)
            if step % accumulation_steps == 0 or step == len(tbar) - 1:
                # Unscales the gradients of optimizer's assigned params in-place
                #    scaler.unscale_(optimizer)

                #                   # Since the gradients of optimizer's assigned params are unscaled, clips as usual:
                #    torch.nn.utils.clip_grad_norm_(net.parameters(), CFG.max_grad_norm)

                scaler.step(optimizer)
                scaler.update()
                # optimizer.step()
                scheduler.step()
                optimizer.zero_grad(set_to_none=True)
                

            avg_loss = np.round(np.mean(loss_list), 4)
            cur_lrs = [float(param_group["lr"]) for param_group in optimizer.param_groups]
            
            pred_val = pred.detach().cpu().numpy()
            btarget_val = batch["labels"].detach().cpu().numpy()
            cur_rr = 1/np.take_along_axis(arr=(pred_val.shape[1]+1 - rankdata(pred_val,axis=1)),indices=btarget_val[:,None],axis=1) 
                
            total_rr.append(cur_rr)
            tbar.set_description(
                f"Train Epoch {epoch + 1} Loss: {avg_loss} RR:{np.mean(total_rr)} lr: { cur_lrs[0]:4f},{ cur_lrs[1]:4f}"
            )

            del pred, batch
            gc.collect()
        total_rr=[]
        torch.cuda.empty_cache()
        model.eval()
        with torch.no_grad():
            avg_val_loss = 0.0
            tbar_val = tqdm(dl_val)
            for step, batch in enumerate(tbar_val):
                batch = {key: part.to(device) for key, part in batch.items()}

                # input_ids = data['ids'].cuda()
                # input_masks = data['mask'].cuda()
                # tag_ids = data['tag_ids'].cuda()
                # targets = data['targets'].cuda()#.long().view(-1).cuda()

                pred = model(batch["input_ids"], attention_mask=batch["attention_mask"])[
                    "logits"
                ]
                # pred = checkpoint(net,(input_ids,input_masks))

                # loss = loss_fn(pred.softmax(dim=1), targets)
                loss = loss_fn(pred, batch["labels"])
                
                val_loss_list.append(loss.detach().cpu().item())
                pred_val = pred.detach().cpu().numpy()
                btarget_val = batch["labels"].detach().cpu().numpy()
                cur_rr = 1/np.take_along_axis(arr=(pred_val.shape[1]+1 - rankdata(pred_val,axis=1)),indices=btarget_val[:,None],axis=1) 
                
                total_rr.append(cur_rr)
                avg_val_loss = np.round(np.mean(val_loss_list), 4)
                tbar_val.set_description(f" Val Epoch {epoch + 1} Loss: {avg_val_loss} RR:{np.mean(cur_rr)}")
            print(avg_val_loss,np.mean(np.concatenate(total_rr)))

            model.save_pretrained(f"saved_model/ranker_{model_name}_{lr}lr_{EPOCHS}/e_{epoch}/f{fold}") #_f{fold}")
            if (avg_val_loss/avg_loss)>1.5:
                break  
            
        # if True:

        #     scheduler2.step(avg_val_loss)

        general_val_loss.append(avg_val_loss)
        # with open('roberta_large_epoch_%s_fold_%s.pkl' % (epoch, i), 'wb') as f: pkl.dump(net.to(torch.device("cpu")), f)
        # torch.save(net.state_dict(),'deberta_base5_epoch_%s_fold_%s.pkl' % (epoch, i))
        torch.cuda.empty_cache()

   

In [140]:
print("============================================================")
print(f" {epochs} -- {lr} -- {datetime.datetime.now()}")
print("============================================================")

# with open(f'{model_name}_train.pkl', 'rb') as f:
#     train_dataset = pickle.load(f)
skf = KFold(FOLDS, random_state=42, shuffle=True)
for fold, (tr_id, val_id) in enumerate(skf.split(dataset_for_cv[0])):
    # dfdev.to_csv('public_data/clarifying_questions_val_w_context.csv', index=False)
    
    dataset_tr = MultiPleChoiceRankerAugmentedTrainDataset([dat[tr_id] for dat in dataset_for_cv])
    dataset_val = MultiPleChoiceRankerDataset(dataset_for_cv[0][val_id])

    from transformers import AutoTokenizer

    tokenizer = AutoTokenizer.from_pretrained(model_hug)

    dl = DataLoader(
    dataset_tr,
    collate_fn=DataCollatorForMultipleChoice(tokenizer, max_length=320),
    batch_size= 3,#8,
    pin_memory=True,
    shuffle=True,
)
    dl_val = DataLoader(
        dataset_val,
        collate_fn=DataCollatorForMultipleChoice(tokenizer, max_length=320),
        batch_size=3,#8,
        pin_memory=True,
    )

    device = "cuda:0"
    from transformers import AutoModelForSequenceClassification

    model = AutoModelForMultipleChoice.from_pretrained("microsoft/deberta-v3-base")
    model.to(device)
    model.load_state_dict(torch.load(f"saved_model/ranker_{old_model_name}_{lr}lr_{epochs}/e_4/f{fold}/pytorch_model.bin"))
    model.gradient_checkpointing_enable()

    optimizer_parameters = get_optimizer_params(
    model,
    encoder_lr=CFG.encoder_lr,
    decoder_lr=CFG.decoder_lr,
    weight_decay=CFG.weight_decay,
)


    optimizer = Adam(optimizer_parameters, lr=CFG.encoder_lr, eps=CFG.eps, betas=CFG.betas)


    # optimizer_grouped_parameters = [{'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01}, {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]

    num_train_steps = int(EPOCHS * len(dl) / accumulation_steps)
    # scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0.3 * num_train_optimization_steps, num_training_steps = num_train_optimization_steps)
    print(num_train_steps)
    scheduler = get_scheduler(CFG, optimizer, num_train_steps)
    criterion = torch.nn.CrossEntropyLoss()
    

    # Train Model
    train_ranker(
        model_name,
        model,
        dl,
        dl_val,
        scheduler,
        optimizer,
        criterion,
        epochs,
        lr,
        fold,
    )
del model
torch.cuda.empty_cache()

 5 -- 7e-05 -- 2022-10-07 23:40:24.572378


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/felipe/anaconda3/envs/iglu2/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:434: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2ForMultipleChoice: ['mask_predictions.LayerNorm.bias', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'l

148


  0%|          | 0/238 [00:00<?, ?it/s]/home/felipe/anaconda3/envs/iglu2/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2339: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/home/felipe/anaconda3/envs/iglu2/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
Train Epoch 1 Loss: 2.0629 RR:0.6116862420128752 lr: 0.000056,0.000056: 100%|█████████▉| 237/238 [15:44<00:03,  4.00s/

2.1451 0.5561356671473406


...

2.2023 0.5865265946115953


Train Epoch 4 Loss: 1.6075 RR:[[0.70022568]90678 lr: 0.000012,0.000012: 100%|█████████▉| 237/238 [23:40<00:05,  5.67s/it]
 [0.67103999]
Train Epoch 4 Loss: 1.6075 RR:[[0.70022568]█████████▉| 237/238 [23:41<00:05,  5.67s/it]
 [0.67103999]
Train Epoch 4 Loss: 1.6075 RR:[[0.70022568]██████████| 238/238 [23:43<00:00,  4.84s/it]
 [0.67103999]
 Val Epoch 4 Loss: 2.1007 RR:0.5: 100%|██████████| 60/60 [01:26<00:00,  1.44s/it]                


2.1007 0.5941716876879327


Train Epoch 5 Loss: 1.5857 RR:[[0.69827662]11481 lr: 0.000000,0.000000: 100%|█████████▉| 237/238 [24:04<00:06,  6.10s/it]
 [0.67579433]
Train Epoch 5 Loss: 1.5857 RR:[[0.69827662]█████████▉| 237/238 [24:07<00:06,  6.10s/it]
 [0.67579433]
Train Epoch 5 Loss: 1.5857 RR:[[0.69827662]██████████| 238/238 [24:08<00:00,  5.38s/it]
 [0.67579433]
 Val Epoch 5 Loss: 2.1149 RR:0.5: 100%|██████████| 60/60 [01:36<00:00,  1.60s/it]                


2.1149 0.592733582107103
